In [ ]:
from llama_parse import LlamaParse
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from groq import Groq
from langchain_groq import ChatGroq
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()
from langchain.chat_models import ChatOpenAI
import chardet
from dotenv import load_dotenv
import streamlit as st



llamaparse_api_key = os.environ.get('LLAMA_CLOUD_API_KEY')
openai_api_key = os.environ.get("OPENAI_API_KEY")



def load_or_parse_data():
    data_file = "./data/parsed_data.pkl"

    if os.path.exists(data_file):
        # Load the parsed data from the file
        parsed_data = joblib.load(data_file)
    else:
        # Perform the parsing step and store the result in llama_parse_documents
        parsingInstructionUber10k = """The provided document is a HR policies
        of an organization.
        Try to be precise while answering the questions"""
        parser = LlamaParse(api_key=llamaparse_api_key,
                            result_type="markdown",
                            parsing_instruction=parsingInstructionUber10k,
                            max_timeout=5000,)
        llama_parse_documents = parser.load_data("./data/HR_Policy_Manual_KFSLnew.pdf")


        # Save the parsed data to a file
        print("Saving the parse results in .pkl format ..........")
        joblib.dump(llama_parse_documents, data_file)

        # Set the parsed data to the variable
        parsed_data = llama_parse_documents

    return parsed_data

def convert_to_utf8(file_path):
    """
    Convert the given file to UTF-8 encoding if it's not already in that format.
    """
    with open(file_path, 'rb') as f:
        raw_data = f.read()

    # Detect file encoding using chardet
    result = chardet.detect(raw_data)
    encoding = result['encoding'] or 'utf-8'  # Default to utf-8 if detection fails
    # print(encoding)

    print(f"Detected encoding: {encoding}")

    # Decode using the detected encoding and re-encode as utf-8
    text = raw_data.decode(encoding)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(text)

    print(f"File converted to UTF-8 successfully: {file_path}")



def create_vector_database():
    """
    Creates a vector database using document loaders and embeddings.

    This function loads urls,
    splits the loaded documents into chunks, transforms them into embeddings using OllamaEmbeddings,
    and finally persists the embeddings into a Chroma vector database.

    """
    # Call the function to either load or parse the data
    llama_parse_documents = load_or_parse_data()
    # print(llama_parse_documents)

    with open('data/output.md', 'a', encoding='utf-8', errors='ignore') as f:  # Open the file in append mode ('a')
        for doc in llama_parse_documents:
            f.write(doc.text + '\n')

    markdown_path = "./data/output.md"

    convert_to_utf8("data/output.md")

    loader = UnstructuredMarkdownLoader(markdown_path)

   #loader = DirectoryLoader('data/', glob="**/*.md", show_progress=True)
    documents = loader.load()
    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    #len(docs)
    print(f"length of documents loaded: {len(documents)}")
    print(f"total number of document chunks generated :{len(docs)}")
    #docs[0]

    # Initialize Embeddings
    embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

    # Create and persist a Chroma vector database from the chunked documents
    vs = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory="chroma_db_llamaparse1",  # Local mode with in-memory storage only
        collection_name="rag"
    )

    print('Vector DB created successfully !')
    return vs,embed_model

llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0
    )

vs,embed_model = create_vector_database()

def instantiate_vectordb(embed_model):
    vectorstore = Chroma(embedding_function=embed_model,
                        persist_directory="chroma_db_llamaparse1",
                        collection_name="rag")

    retriever=vectorstore.as_retriever(search_kwargs={'k': 1})
    return retriever

retriever = instantiate_vectordb(embed_model)

def set_custom_prompt():

    custom_prompt_template = """Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}
    Question: {question}

    Only return the helpful answer below in a complete sentence and nothing else.
    format the answers in bullets wherever required and prettify the text. Mention all the nested points that are truly required.
    Helpful answer:
    """

    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

prompt = set_custom_prompt()
print(prompt)

memory = ConversationBufferMemory(memory_key='result', output_key='result', return_messages=True)

conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={'prompt':prompt}
    )

qa = RetrievalQA.from_chain_type(llm=llm,
                               chain_type="stuff",
                               retriever=retriever,
                               return_source_documents=True,
                               memory = memory,
                               chain_type_kwargs={"prompt": prompt})

def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    # for i, message in enumerate(st.session_state.chat_history):
    #     if i % 2 == 0:
    #         st.write(user_template.replace(
    #             "{{MSG}}", message.content), unsafe_allow_html=True)
    #     else:
    #         st.write(bot_template.replace(
    #             "{{MSG}}", message.content), unsafe_allow_html=True)
            

def main():
    load_dotenv()

    st.set_page_config(page_title="HR Assistant",page_icon=":books:")
    # st.write(css, unsafe_allow_html=True)

    st.subheader("HR Assistant :books:")

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    user_question = st.chat_input("Ask anything here:")
    if user_question:
        handle_userinput(user_question)

    st.session_state.conversation = conversation_chain

            
if __name__ == '__main__':
    main()